<a href="https://colab.research.google.com/github/Dhanapal-Angamuthu/DeepLearning/blob/main/CNN/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow import keras


In [6]:
## Lets use keras inbuild fashion mnist dataset
fashion_mnist = keras.datasets.fashion_mnist

In [7]:
(train_img, train_label), (test_img, test_label) = fashion_mnist.load_data()


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
#Scale down grey scan images into 0 or 1 for easy processing
train_img = train_img/255.0
test_img = test_img/255.0

In [13]:
train_img[0].shape

(28, 28)

In [15]:
train_img = train_img.reshape(len(train_img), 28, 28, 1)
test_img = test_img.reshape(len(test_img), 28, 28, 1)

In [21]:
def build_model(hp):
  model = keras.Sequential([
      keras.Input(shape=(28, 28, 1)), # Add Input layer here
      keras.layers.Conv2D(
          filters = hp.Int('conv_filter1', min_value = 32, max_value = 128, step = 16),
          kernel_size = hp.Choice('conv_kernel1', values = [3,5]),
          activation = 'relu',
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_filter2', min_value = 32, max_value = 64, step = 16),
          kernel_size = hp.Choice('conv_kernel2', values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_units', min_value = 32, max_value = 128, step = 16),
          activation = 'relu'
      ),
      keras.layers.Dense(10, activation = 'softmax')
  ])
  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [22]:
import keras_tuner
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [23]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5,
                            directory = 'output', project_name = 'fashion_mnist')

In [24]:
tuner_search.search(train_img, train_label, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 40s]
val_accuracy: 0.8498333096504211

Best val_accuracy So Far: 0.8498333096504211
Total elapsed time: 00h 02m 45s


In [26]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 112)    │         1,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 32)     │        89,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 112)            │     1,734,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,826,650 (6.97 MB)

 Trainable params: 1,826,650 (6.97 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# Recompile the model after loading
model.compile(optimizer = keras.optimizers.Adam(), # You can use the same optimizer or a new one
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 112)    │         1,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 32)     │        89,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 112)            │     1,734,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,826,650 (6.97 MB)

 Trainable params: 1,826,650 (6.97 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.fit(train_img, train_label, epochs = 10, validation_split = 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8526 - loss: 0.4052 - val_accuracy: 0.8482 - val_loss: 0.4197
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8662 - loss: 0.3710 - val_accuracy: 0.8673 - val_loss: 0.3681
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8748 - loss: 0.3378 - val_accuracy: 0.8745 - val_loss: 0.3378
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8849 - loss: 0.3153 - val_accuracy: 0.8817 - val_loss: 0.3221
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8903 - loss: 0.2982 - val_accuracy: 0.8705 - val_loss: 0.3604
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8977 - loss: 0.2797 - val_accuracy: 0.8875 - val_loss: 0.3175
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9023 - loss: 0.2607 - val_accuracy: 0.8843 - val_loss: 0.3110
